In [21]:
import networkx as nx
import numpy as np
import qldpc
from qldpc import codes
from qldpc.objects import Pauli
import cudaq
import cudaq_qec as qec
from sympy.abc import x, y

Creating a custom code

In [27]:
dims = {x: 12, y: 6}
poly_a = x**3 + y + y**2
poly_b = y**3 + x + x**2

code = qldpc.codes.BBCode(dims, poly_a, poly_b)

print(code)
print()
print("number of logical qubits:", code.dimension)

print("code distance: <=", code.get_distance_bound(num_trials=100))

BBCode on 144 qubits with cyclic group orders {x: 12, y: 6} and generating polynomials
  A = x**3 + y**2 + y
  B = x**2 + x + y**3

number of logical qubits: 12
code distance: <= 12


In [24]:
code.matrix.shape

(360, 720)

In [25]:
distance = 3
code = qldpc.codes.SurfaceCode(distance, rotated=True)

In [26]:
code.matrix.shape

(8, 18)

In [16]:
stabs = []
for row in code.matrix_x:
    stab = ["X" if i == 1 else "I" for i in row]
    stabs.append("".join(stab))

for row in code.matrix_z:
    stab = ["Z" if i == 1 else "I" for i in row]
    stabs.append("".join(stab))
    

In [17]:
code.get_logical_ops()

GF([[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]], order=2)

In [ ]:
@cudaq.kernel
def prep0(logicalQubit: patch):
    h(logicalQubit.data[0], logicalQubit.data[4], logicalQubit.data[6])
    x.ctrl(logicalQubit.data[0], logicalQubit.data[1])
    x.ctrl(logicalQubit.data[4], logicalQubit.data[5])
    # ... additional initialization gates ...

@cudaq.kernel
def stabilizer(logicalQubit: patch,
              x_stabilizers: list[int],
              z_stabilizers: list[int]) -> list[bool]:
    # Measure X stabilizers
    h(logicalQubit.ancx)
    for xi in range(len(logicalQubit.ancx)):
        for di in range(len(logicalQubit.data)):
            if x_stabilizers[xi * len(logicalQubit.data) + di] == 1:
                x.ctrl(logicalQubit.ancx[xi], logicalQubit.data[di])
    h(logicalQubit.ancx)

    # Measure Z stabilizers
    for zi in range(len(logicalQubit.ancx)):
        for di in range(len(logicalQubit.data)):
            if z_stabilizers[zi * len(logicalQubit.data) + di] == 1:
                x.ctrl(logicalQubit.data[di], logicalQubit.ancz[zi])

    # Get and reset ancillas
    results = mz(logicalQubit.ancz, logicalQubit.ancx)
    reset(logicalQubit.ancx)
    reset(logicalQubit.ancz)
    return results

In [ ]:
@qec.code('bbcode')
class BBCode:
    def __init__(self, **kwargs):
        qec.Code.__init__(self, **kwargs)

        # Define stabilizer generators
        self.stabilizers = qec.Stabilizers([stabs])

        # Register quantum kernels
        self.operation_encodings = {
            qec.operation.prep0: prep0,
            qec.operation.stabilizer_round: stabilizer
        }

    def get_num_data_qubits(self):
        return code.num_qubits

    def get_num_ancilla_x_qubits(self):
        return code.num_checks_x

    def get_num_ancilla_z_qubits(self):
        return code.num_checks_z

    def get_num_ancilla_qubits(self):
        return code.num_checks

In [11]:
# Create code and decoder
code = qec.get_code('steane')
decoder = qec.get_decoder('single_error_lut',
                          code.get_parity())

# Get a QEC code
cudaq.set_target("stim")
steane = qec.get_code("steane")

# Get the parity check matrix of a code
# Can get the full code, or for CSS codes
# just the X or Z component
H = steane.get_parity()
print(f"H:\n{H}")
observables = steane.get_pauli_observables_matrix()
Lz = steane.get_observables_z()
print(f"observables:\n{observables}")
print(f"Lz:\n{Lz}")

nShots = 3
nRounds = 4

# error probabily
p = 0.01
noise = cudaq.NoiseModel()
noise.add_all_qubit_channel("x", qec.TwoQubitDepolarization(p), 1)

# prepare logical |0> state, tells the sampler to do z-basis experiment
statePrep = qec.operation.prep0
# our expected measurement in this state is 0
expected_value = 0

# sample the steane memory circuit with noise on each cx gate
# reading out the syndromes after each stabilizer round (xor'd against the previous)
# and readout out the data qubits at the end of the experiment
syndromes, data = qec.sample_memory_circuit(steane, statePrep, nShots, nRounds,
                                            noise)
print("From sample function:\n")
print("syndromes:\n", syndromes)
print("data:\n", data)

# Get a decoder
decoder = qec.get_decoder("single_error_lut", H)
nLogicalErrors = 0

# Logical Mz each shot (use Lx if preparing in X-basis)
logical_measurements = (Lz @ data.transpose()) % 2
# only one logical qubit, so do not need the second axis
logical_measurements = logical_measurements.flatten()
print("LMz:\n", logical_measurements)

# initialize a Pauli frame to track logical flips
# through the stabilizer rounds
pauli_frame = np.array([0, 0], dtype=np.uint8)
for shot in range(0, nShots):
    print("shot:", shot)
    for syndrome in syndromes:
        print("syndrome:", syndrome)
        # decode the syndrome
        decoder_result = decoder.decode(syndrome)
        convergence, result = decoder_result.converged, decoder_result.result
        data_prediction = np.array(result, dtype=np.uint8)

        # see if the decoded result anti-commutes with the observables
        print("decode result:", data_prediction)
        decoded_observables = (observables @ data_prediction) % 2
        print("decoded_observables:", decoded_observables)

        # update pauli frame
        pauli_frame = (pauli_frame + decoded_observables) % 2
        print("pauli frame:", pauli_frame)

    # after pauli frame has tracked corrections through the rounds
    # apply the pauli frame correction to the measurement, and see
    # if this matches the state we intended to prepare
    # We prepared |0>, so we check if logical measurement Mz + Pf_X = 0
    corrected_mz = (logical_measurements[shot] + pauli_frame[0]) % 2
    print("Expected value:", expected_value)
    print("Corrected value:", corrected_mz)
    if (corrected_mz != expected_value):
        nLogicalErrors += 1

# Count how many shots the decoder failed to correct the errors
print("Number of logical errors:", nLogicalErrors)

H:
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 1 1]]
observables:
[[0 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1]]
Lz:
[[0 0 0 0 1 1 1]]
From sample function:

syndromes:
 [[0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 1 1 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
data:
 [[1 0 0 0 0 1 1]
 [0 1 0 1 1 0 1]
 [0 0 0 1 0 1 1]]
LMz:
 [0 0 0]
shot: 0
syndrome: [0 1 0 0 0 0]
decode result: [0 0 0 0 1 0 0 0 0 0 0 0 0 0]
decoded_observables: [1 0]
pauli frame: [1 0]
syndrome: [1 0 0 0 0 0]
decode result: [1 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded_observables: [0 0]
pauli frame: [1 0]
syndrome: [0 0 0 0 0 0]
decode result: [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded_observables: [0 0]
pauli frame: [1 0]
syndrome: [0 0 0 0 0 0]
decode result: [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
decoded_observables: [0 0]
pauli frame: [1 0]


In [10]:
help(result)

Help on DecoderResult in module cudaq_qec._pycudaqx_qec_the_suffix_matters_cudaq_qec.qecrt object:

class DecoderResult(pybind11_builtins.pybind11_object)
 |  A class representing the results of a quantum error correction decoding operation.
 |  
 |  This class encapsulates both the convergence status and the actual decoding result.
 |  
 |  Method resolution order:
 |      DecoderResult
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: cudaq_qec._pycudaqx_qec_the_suffix_matters_cudaq_qec.qecrt.DecoderResult) -> None
 |      
 |      
 |      Default constructor for DecoderResult.
 |      
 |      Creates a new DecoderResult instance with default values.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  converged
 |      Boolean flag indicating if the decoder converged to a solution.
 |      
 |      True if the decoder s

In [8]:
import numpy as np
import cudaq
import cudaq_qec as qec

# Get a QEC code
cudaq.set_target("stim")
steane = qec.get_code("steane")

# Get the parity check matrix of a code
# Can get the full code, or for CSS codes
# just the X or Z component
H = steane.get_parity()
print(f"H:\n{H}")
observables = steane.get_pauli_observables_matrix()
Lz = steane.get_observables_z()
print(f"observables:\n{observables}")
print(f"Lz:\n{Lz}")

nShots = 3
nRounds = 4

# error probabily
p = 0.01
noise = cudaq.NoiseModel()
noise.add_all_qubit_channel("x", qec.TwoQubitDepolarization(p), 1)

# prepare logical |0> state, tells the sampler to do z-basis experiment
statePrep = qec.operation.prep0
# our expected measurement in this state is 0
expected_value = 0

# sample the steane memory circuit with noise on each cx gate
# reading out the syndromes after each stabilizer round (xor'd against the previous)
# and readout out the data qubits at the end of the experiment
syndromes, data = qec.sample_memory_circuit(steane, statePrep, nShots, nRounds,
                                            noise)
print("From sample function:\n")
print("syndromes:\n", syndromes)
print("data:\n", data)

# Get a decoder
decoder = qec.get_decoder("single_error_lut", H)
nLogicalErrors = 0

# Logical Mz each shot (use Lx if preparing in X-basis)
logical_measurements = (Lz @ data.transpose()) % 2
# only one logical qubit, so do not need the second axis
logical_measurements = logical_measurements.flatten()
print("LMz:\n", logical_measurements)

# initialize a Pauli frame to track logical flips
# through the stabilizer rounds
pauli_frame = np.array([0, 0], dtype=np.uint8)
for shot in range(0, nShots):
    print("shot:", shot)
    for syndrome in syndromes:
        print("syndrome:", syndrome)
        # decode the syndrome
        convergence, result = decoder.decode(syndrome)
        data_prediction = np.array(result, dtype=np.uint8)

        # see if the decoded result anti-commutes with the observables
        print("decode result:", data_prediction)
        decoded_observables = (observables @ data_prediction) % 2
        print("decoded_observables:", decoded_observables)

        # update pauli frame
        pauli_frame = (pauli_frame + decoded_observables) % 2
        print("pauli frame:", pauli_frame)

    # after pauli frame has tracked corrections through the rounds
    # apply the pauli frame correction to the measurement, and see
    # if this matches the state we intended to prepare
    # We prepared |0>, so we check if logical measurement Mz + Pf_X = 0
    corrected_mz = (logical_measurements[shot] + pauli_frame[0]) % 2
    print("Expected value:", expected_value)
    print("Corrected value:", corrected_mz)
    if (corrected_mz != expected_value):
        nLogicalErrors += 1

# Count how many shots the decoder failed to correct the errors
print("Number of logical errors:", nLogicalErrors)

H:
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 1 1]]
observables:
[[0 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1]]
Lz:
[[0 0 0 0 1 1 1]]
From sample function:

syndromes:
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]]
data:
 [[0 1 1 0 1 1 0]
 [0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1]]
LMz:
 [0 0 1]
shot: 0
syndrome: [0 0 0 0 0 0]


TypeError: cannot unpack non-iterable cudaq_qec._pycudaqx_qec_the_suffix_matters_cudaq_qec.qecrt.DecoderResult object